In [1]:
import requests

In [2]:
# get response
# url_string = "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C44&q=application+of+machine+learning+in+cancer+detection&btnG="
url_string = "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C44&q=nanoparticles+in+cancer+ug+discovry&btnG="
result = requests.get(url_string)

In [3]:
type(result)

requests.models.Response

In [4]:
# calling .text method on the result variable returns the html document
result.content

b'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">\n<html>\n<head><meta http-equiv="content-type" content="text/html; charset=utf-8"><meta name="viewport" content="initial-scale=1"><title>https://scholar.google.com/scholar?hl=en&amp;as_sdt=0%2C44&amp;q=nanoparticles+in+cancer+ug+discovry&amp;btnG=</title></head>\n<body style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px; overscroll-behavior:contain;" onload="e=document.getElementById(\'captcha\');if(e){e.focus();} if(solveSimpleChallenge) {solveSimpleChallenge(,);}">\n<div style="max-width:400px;">\n<hr noshade size="1" style="color:#ccc; background-color:#ccc;"><br>\n<div style="font-size:13px;">\nOur systems have detected unusual traffic from your computer network.  Please try your request again later.  <a href="#" onclick="document.getElementById(\'infoDiv0\').style.display=\'block\';">Why did this happen?</a><br><br>\n<div id="infoDiv0" style="display:none; backgro

### Getting 'soup' object for scraping

In [48]:
# to extract specific information from the text file we can use bs4
import bs4

In [49]:
soup = bs4.BeautifulSoup(result.content, "lxml")

In [50]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="initial-scale=1" name="viewport"/>
  <title>
   https://scholar.google.com/scholar?start=10&amp;q=machine+learning+in+cancer+detection&amp;hl=en&amp;as_sdt=0,44
  </title>
 </head>
 <body onload="e=document.getElementById('captcha');if(e){e.focus();} if(solveSimpleChallenge) {solveSimpleChallenge(,);}" style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px; overscroll-behavior:contain;">
  <div style="max-width:400px;">
   <hr noshade="" size="1" style="color:#ccc; background-color:#ccc;"/>
   <br/>
   <div style="font-size:13px;">
    Our systems have detected unusual traffic from your computer network.  Please try your request again later.
    <a href="#" onclick="document.getElementById('infoDiv0').style.display='block';">
     Why did this happen?
    </a>
    <br/>
  

### Grabbing search item 'title'

In [51]:
# each result is encapsulated by a [data-lid] tag
soup.select('[data-lid]')[0]    # returns a list

IndexError: list index out of range

In [ ]:
soup.select('[data-lid]')[0].select('h3')[0].get_text()

In [ ]:
# getting all search result titles from the page
res_list = list(soup.select('[data-lid]'))
len(res_list)

In [ ]:
# creating a list of titles
titles = [item.select('h3')[0].get_text() for item in res_list]
titles

### Grabbing search query entered by user

In [ ]:
# grabbing search query for stroring search data

search_query_list = []
res_set = soup.select('.gs_in_txt')
for i in range(0, int(len(res_set)/2)):
    search_query_list.append(res_set[i]['value'])
    
''.join(search_query_list)

### Grabbing url to each search result

In [ ]:
# grabbing url
res_list = list(soup.select('[data-lid]'))
article_urls = [item.find_all('a')[1]['href'] for item in res_list]
article_urls

In [ ]:
res_list = list(soup.select('[data-lid]'))
article_urls = []
for item in res_list:
    if 'https:' in item.find_all('a')[1]['href']:
        article_urls.append(item.find_all('a')[1]['href'])
    else:
        article_urls.append(item.find_all('a')[0]['href'])
        
for url in article_urls:
    print(url)

In [ ]:
url = "https://www.researchgate.net/profile/Raed-Shubair-2/publication/320046589_Comparative_Study_of_Machine_Learning_Algorithms_for_Breast_Cancer_Detection_and_Diagnosis/links/59caded3aca272bb05079958/Comparative-Study-of-Machine-Learning-Algorithms-for-Breast-Cancer-Detection-and-Diagnosis.pdf"
len(url[0:255])

### Grabbing author names

In [ ]:
soup.select('.gs_a')[0].find_all('a')

In [ ]:
# grabbing author names
auth_list = []
for author in soup.select('.gs_a')[2].find_all('a'):
    auth_list.append(author.get_text())
    
', '.join(auth_list)   # A string of author names

In [ ]:
# getting all first authors using list comprehension
res_list = soup.select('.gs_a')
first_authors_list = [item.find_all('a')[0].get_text() for item in res_list]
first_authors_list

### Grabbing publication year

In [ ]:
# grabbing year
for value in soup.select('.gs_a')[2].get_text().split():
    if value.isnumeric():
        print(value)

In [ ]:
res_list = list(soup.select('.gs_a'))
years_list = []
    
for value in res_list:
    terms = value.get_text().split()
    value = [item for item in terms if item.isnumeric()]
    years_list.append(value[0])

years_list

### Grabbing name of the journal

In [ ]:
# journal name as it is typically the last item under the '.gs_a' class
soup.select('.gs_a')[2].get_text().split()[-1]

In [ ]:
res_list = list(soup.select('.gs_a'))
journal_names_list = [item.get_text().split()[-1] for item in res_list]
journal_names_list

### Grabbing total number of citations for each result

In [ ]:
# the 'gs_fl' class containing citation for each search item = (i*2)+1 ( i = 0, the index is 1)
res_list = list(soup.select('.gs_fl'))
valid_res_list = [item for item in res_list if len(item["class"]) == 1]
valid_res_list

In [ ]:
valid_res_list[0].find_all('a')[2].get_text()

In [ ]:
total_citations_list = [int(item.find_all('a')[2].get_text().split()[-1]) for item in valid_res_list]
total_citations_list